# Preambolo
prova a considerare class weight balanced.
Fai albero ottimo 3 vs altri, 4 vs altri e fai il pdf-->10 iterazioni, ricordati di fare holdout
Per classe 4  fai split 70-30.
Tumori maligni --> 1 carcinoma, 2 linfoma , 5 è cisto adenoma ma non se se de preciso così e se è benigno o maligno.
Per ora 5 lo considero poco, è una classe un po' indeterminata

1,2 vs 3,4,5--> maligni contro benigni

f1-score--> meglio dell'accuratezza per classi non bilanciate
tu cerca di aggirare lo sbolanciamento tra classi--> nell'allenamento degli alberi considero lo weight pesando più le classi meno rappresentate-->peso inversamente proporzionale al numero

nella media dell'accuratezza visualizza anche dev std


## Moduli importati

In [1]:
import graphviz
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTENC, SMOTE
from sklearn import tree
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, make_scorer, precision_score

## Importo dataset

In [2]:
df = pd.read_excel('../Db_parotide_Def_REV_Samuel.xlsx')
df['ADC'] = df['ADC'].apply(lambda s: str(s).replace(',', '.')).astype(float)
df.iat[0, 6] = 3
df.iat[1, 6] = 3
df.iat[20, 6] = 3
df.head()

,ID,eta,Segni macro malignità,ADC,TIC type,T2,Output Algoritmo,COD ISTO,ISTO DEFINITIVO-PULITO,ISTO DEFINITIVO,...,CONDORD CODICE FNAC-ISTO DIAGN,CONCORD multiRMN-ISTO B/M,CONCORD multiRMN-ISTO DIAGN,CONCORD FNAC-RMN,FNAC,DATA FNAC,ESITO FNAC,COD FNAC,MILAN,DATA INTERVENTO
0,4,33,0,2.60,D,0,3,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,27/12/2017,Positivo per CTM. Carcinoma squamocellulare o ...,3,6,24/01/2018
1,10,50,0,2.44,D,0,3,1,CARCINOMA,CARCINOMA ADENOIDECISTICO G3,...,1,1,1,1,0,27/12/2016,carcinoma adenoidecistico,3,6,27/11/2019
2,9,81,1,0.70,C,1,1,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,21/03/2019,"Positivo per CTM, carcinoma squamocellulare",3,6,15/05/2019
3,1,28,0,1.60,A,1,3,1,CARCINOMA,CARCINOMA DEI DOTTI SALIVARI,...,1,1,1,1,0,2016-06-10 00:00:00,sospetto per CTM,5,5,25/01/2017
4,7,51,0,1.40,A,0,3,1,METASTASI DA CARCINOMA SQUAMOCELLULARE ALLA PA...,METASTASI DA CARCINOMA SQUAMOCELLULARE ALLA PA...,...,1,1,1,1,0,13/09/2018,Positivo per CTM. Carcinoma squamocellulare,3,6,2018-10-10 00:00:00


## Samples e labels

In [3]:
X_not_encoded = df.loc[:,'eta':'T2'].copy()
X = pd.get_dummies(X_not_encoded, columns = ['TIC type'])
y = df['COD ISTO'].copy()
y.value_counts()


3    54
4    27
1    16
2     5
5     5
Name: COD ISTO, dtype: int64

## Funzione

In [4]:
#qui faccio iterazioni di stratified k fold, splitto stratificando, augmento il training set, faccio augmentation con il training set
#prima di rendere le labels binarie, calcolo una score sul test set e alla fine restituisco la media delle score 
#con la deviaizione std


#assumo di passargli X not encode e y
#se scegli smote e non smote sarò io a fare la one hot encoding 

def AUG(X, y, target_one, tuned_parameters, aug_method = 'smotenc',
                scoring_func = accuracy_score, n_splits= 3,
                random_state_tree = None,
                n_it = 10, filename = 'albero'):

    scoring = make_scorer(scoring_func)
    avg_scores = []
    models_ = []
    
    for n in range(n_it):
    
        print('### ITERATION %d ###\n\n' %(n+1))

        scores = []
        models = []

        skf = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = None)

        for train_index, test_index in skf.split(X, y):

            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            #sempre che con smote faccio oversampling di tutte le classi non maggioritarie 
            if aug_method == 'smotenc':#assumo tutte features categoriche 
                sm = SMOTENC(random_state = None, k_neighbors = 2,
                                                   categorical_features = [3])
                X_train_aug, y_train_aug = sm.fit_resample(X_train, y_train)
                X_train_aug = pd.get_dummies(X_train_aug, columns = ['TIC type'])
                
                

            elif aug_method == 'smote':#devo fare one hot encoding 
                X_train = pd.get_dummies(X_train, columns = ['TIC type'])
                sm = SMOTE(random_state = None, k_neighbors = 2)
                X_train_aug, y_train_aug = sm.fit_resample(X_train, y_train)
            

            y_train_aug_bin = [1 if x in target_one else 0 for x in y_train_aug]
            y_test_bin = [1 if x in target_one else 0 for x in y_test]
            X_test = pd.get_dummies(X_test, columns = ['TIC type'])
            
            if 'TIC type_A' not in X_train_aug.columns:
                X_train_aug['TIC type_A'] = 0
            if 'TIC type_B' not in X_train_aug.columns:
                X_train_aug['TIC type_B'] = 0
            if 'TIC type_C' not in X_train_aug.columns:
                X_train_aug['TIC type_C'] = 0
            if 'TIC type_D' not in X_train_aug.columns:
                X_train_aug['TIC type_D'] = 0
            
            if 'TIC type_A' not in X_test.columns:
                X_test['TIC type_A'] = 0
            if 'TIC type_B' not in X_test.columns:
                X_test['TIC type_B'] = 0
            if 'TIC type_C' not in X_test.columns:
                X_test['TIC type_C'] = 0
            if 'TIC type_D' not in X_test.columns:
                X_test['TIC type_D'] = 0

            albero = tree.DecisionTreeClassifier(random_state = random_state_tree)

            #la famo la gridsearch?? Si daje
            clf = GridSearchCV(albero, tuned_parameters, scoring = scoring,
                               cv = StratifiedKFold(n_splits = 2, shuffle = True, random_state = None),
                               refit = True)
            clf.fit(X_train_aug, y_train_aug_bin)
            
            models.append(clf.best_estimator_)

            params = clf.best_params_
            score = clf.best_score_


            print("Parameters set found:\n")
            print(params)
            print()
            print("Mean cross-validated score of the estimator: %0.3f\n" %(score))
            print()
            current_score = scoring_func(y_test_bin, clf.predict(X_test))
            print('Score computed on the test set: %0.3f\n' %(current_score))
            print('-'*100)

            scores.append(current_score)
            
        avg_scores.append(np.mean(scores))
        models_.append(models[np.argmax(scores)])
        
    print('\n\n# Final Mean score: %0.3f\n# Std dev: %0.3f' %(np.mean(avg_scores), np.std(avg_scores)))
    
    best_model = models_[np.argmax(avg_scores)]
    
    data = tree.export_graphviz(best_model, out_file=None, 
                      feature_names=X_train_aug.columns,  
                      class_names=['0','1'],  
                      filled=True, rounded=True,  
                      special_characters=True) 
    graph = graphviz.Source(data) 
    graph.render('SMOTE-3fold-'+filename)
    

In [5]:
seed = 42 
tuned_parameters = {'criterion': ['gini', 'entropy'],
                    'max_depth': list(range(2,11)),     
                    'min_samples_leaf': list(range(2,11)),
                    'class_weight': ['balanced', None]}

In [6]:
AUG(X_not_encoded, y, [3], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '3-acc', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.828


Score computed on the test set: 0.556

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 8}

Mean cross-validated score of the estimator: 0.817


Score computed on the test set: 0.500

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.828


Score computed on the test set: 0.571

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight': None, 'criterion

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.872


Score computed on the test set: 0.444

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.822


Score computed on the test set: 0.583

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.850


Score computed on the test set: 0.571

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.526
# Std dev: 0.034


In [7]:
AUG(X_not_encoded, y, [3], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '3-f1', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.530


Score computed on the test set: 0.545

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.521


Score computed on the test set: 0.250

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 7}

Mean cross-validated score of the estimator: 0.624


Score computed on the test set: 0.579

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight'

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.518


Score computed on the test set: 0.700

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.596


Score computed on the test set: 0.513

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.517


Score computed on the test set: 0.579

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.536
# Std dev: 0.064


In [8]:
AUG(X_not_encoded, y, [3], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '3-acc-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.844


Score computed on the test set: 0.500

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 10}

Mean cross-validated score of the estimator: 0.822


Score computed on the test set: 0.583

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.822


Score computed on the test set: 0.600

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight': None, 'criterio

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 9}

Mean cross-validated score of the estimator: 0.817


Score computed on the test set: 0.556

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.806


Score computed on the test set: 0.528

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.839


Score computed on the test set: 0.571

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.511
# Std dev: 0.029


In [9]:
AUG(X_not_encoded, y, [3], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '3-f1-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.532


Score computed on the test set: 0.625

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.545


Score computed on the test set: 0.585

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.447


Score computed on the test set: 0.529

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'clas

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.445


Score computed on the test set: 0.588

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.569


Score computed on the test set: 0.605

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 10}

Mean cross-validated score of the estimator: 0.504


Score computed on the test set: 0.154

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.526
# Std dev: 0.040


In [10]:
AUG(X_not_encoded, y, [4], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '4-acc', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.872


Score computed on the test set: 0.750

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.783


Score computed on the test set: 0.750

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.811


Score computed on the test set: 0.714

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight': None, 'criter

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.811


Score computed on the test set: 0.639

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 9}

Mean cross-validated score of the estimator: 0.800


Score computed on the test set: 0.667

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.828


Score computed on the test set: 0.686

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.687
# Std dev: 0.028


In [11]:
AUG(X_not_encoded, y, [4], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '4-f1', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.580


Score computed on the test set: 0.200

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.465


Score computed on the test set: 0.133

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.465


Score computed on the test set: 0.211

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight': 'balanced'

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.445


Score computed on the test set: 0.400

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.626


Score computed on the test set: 0.154

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.404


Score computed on the test set: 0.417

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.283
# Std dev: 0.046


In [12]:
AUG(X_not_encoded, y, [4], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '4-acc-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.828


Score computed on the test set: 0.722

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.811


Score computed on the test set: 0.667

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.844


Score computed on the test set: 0.743

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight': None, 'criterion

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 7}

Mean cross-validated score of the estimator: 0.833


Score computed on the test set: 0.722

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.850


Score computed on the test set: 0.556

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.794


Score computed on the test set: 0.657

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.647
# Std dev: 0.047


In [13]:
AUG(X_not_encoded, y, [4], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '4-f1-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.634


Score computed on the test set: 0.400

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.544


Score computed on the test set: 0.375

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.547


Score computed on the test set: 0.471

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight': 'balanced'

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.455


Score computed on the test set: 0.258

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.538


Score computed on the test set: 0.182

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.492


Score computed on the test set: 0.200

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.300
# Std dev: 0.060


In [14]:
AUG(X_not_encoded, y, [1,2], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '12-acc', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.789


Score computed on the test set: 0.667

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.794


Score computed on the test set: 0.444

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.750


Score computed on the test set: 0.714

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight': None, '

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.717


Score computed on the test set: 0.583

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.789


Score computed on the test set: 0.722

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.706


Score computed on the test set: 0.657

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.625
# Std dev: 0.048


In [15]:
AUG(X_not_encoded, y, [1,2], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '12-f1', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.797


Score computed on the test set: 0.381

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.795


Score computed on the test set: 0.273

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.668


Score computed on the test set: 0.500

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight'

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.732


Score computed on the test set: 0.286

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.799


Score computed on the test set: 0.320

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.625


Score computed on the test set: 0.348

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.255
# Std dev: 0.088


In [16]:
AUG(X_not_encoded, y, [1,2], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '12-acc-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.833


Score computed on the test set: 0.611

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 9, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.772


Score computed on the test set: 0.667

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.778


Score computed on the test set: 0.800

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight': 'balanced', '

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.767


Score computed on the test set: 0.639

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.767


Score computed on the test set: 0.806

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.700


Score computed on the test set: 0.714

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.679
# Std dev: 0.039


In [17]:
AUG(X_not_encoded, y, [1,2], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '12-f1-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.803


Score computed on the test set: 0.267

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.777


Score computed on the test set: 0.222

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.693


Score computed on the test set: 0.190

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


Parameters set found:

{'class_weight': 'balanced'

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.782


Score computed on the test set: 0.235

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.836


Score computed on the test set: 0.286

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.733


Score computed on the test set: 0.125

----------------------------------------------------------------------------------------------------


# Final Mean score: 0.223
# Std dev: 0.058
